In [1]:
import africanus
import dask

In [2]:
from daskms import xds_from_ms
dadasets = xds_from_ms('/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/01.ms', chunks={'row': 400000})

In [3]:
dadasets

[<xarray.Dataset>
 Dimensions:         (chan: 130, corr: 4, row: 8020012, uvw: 3)
 Coordinates:
     ROWID           (row) int32 dask.array<chunksize=(400000,), meta=np.ndarray>
 Dimensions without coordinates: chan, corr, row, uvw
 Data variables:
     ANTENNA1        (row) int32 dask.array<chunksize=(400000,), meta=np.ndarray>
     ANTENNA2        (row) int32 dask.array<chunksize=(400000,), meta=np.ndarray>
     ARRAY_ID        (row) int32 dask.array<chunksize=(400000,), meta=np.ndarray>
     CORRECTED_DATA  (row, chan, corr) complex64 dask.array<chunksize=(400000, 130, 4), meta=np.ndarray>
     DATA            (row, chan, corr) complex64 dask.array<chunksize=(400000, 130, 4), meta=np.ndarray>
     EXPOSURE        (row) float64 dask.array<chunksize=(400000,), meta=np.ndarray>
     FEED1           (row) int32 dask.array<chunksize=(400000,), meta=np.ndarray>
     FEED2           (row) int32 dask.array<chunksize=(400000,), meta=np.ndarray>
     FLAG            (row, chan, corr) bool das

In [4]:
from daskms import xds_to_table

In [5]:
dadasets[0].DATA.data = dadasets[0].MODEL_DATA.data
write = xds_to_table(dadasets, '/lustre/yuping/2020-12-28-dsa2000-take2/test-gain/01.ms', 'DATA')

In [6]:
%time dask.compute(write)

CPU times: user 34.2 s, sys: 2min 19s, total: 2min 53s
Wall time: 6min 37s


(<xarray.Dataset>
 Dimensions:  (chan: 1, corr: 1, row: 21)
 Dimensions without coordinates: chan, corr, row
 Data variables:
     DATA     (row, chan, corr) bool True True True True ... True True True True,)